### Jotting

* [number types](https://numpy.org/devdocs/user/basics.types.html).
* [plotly 3D](https://plotly.com/python/3d-charts/)
* [plotly documentation](https://plotly.github.io/plotly.py-docs/index.html)
* [plotly fig layout](https://plotly.com/python/reference/layout/#layout-title)
* [генератор политры цвета](https://coolors.co/092327-0b5351-00a9a5-4e8098-90c2e7)

# Data and modules loading

### Modules loading

In [ ]:
import pandas as pd
import numpy as np

from plotly.express import histogram, line

import tensorflow_probability
from src.em.mixture import Mixture, DynamicMixture


# Позволяет использовать измененные модули без перезагрузки ядра
%load_ext autoreload
%autoreload 2

### Data loading 

In [ ]:
data_clean = pd.read_csv("nice_jan_march.csv")
data_clean.head()

In [ ]:
# names = [
#     'Year',
#     'Day',
#     'Hour',
#     'Minute',
#     'Bx',
#     'By',
#     'Bz',
#     'Vx',
#     'Vy',
#     'Vz'
# ]
# dataframe = pd.read_table('Data/magfield23.acs', sep=' +', header=None, names=names)
# dataframe.head()
# dataframe.to_csv('Data/magfield23.csv', index=False)
# from auxiliary import time_related_id
# time_related_id(dataframe)
dataframe = pd.read_csv("Data/magfield23_ydhm.csv", na_values=["99999.9"])

In [ ]:
dataframe

In [ ]:
def increm(arr):
    """
    Calculate increments of given array.
    First value is NaN by default
    """
    new_ar = [None]
    for i in range(1, len(arr)):
        inc = arr[i] - arr[i - 1]
        new_ar.append(inc)
    return new_ar


dataframe["dBx"] = increm(dataframe["Bx"].values)
dataframe["dBy"] = increm(dataframe["By"].values)
dataframe["dBz"] = increm(dataframe["Bz"].values)
dataframe["dVx"] = increm(dataframe["Vx"].values)
dataframe["dVy"] = increm(dataframe["Vy"].values)
dataframe["dVz"] = increm(dataframe["Vz"].values)

# Choosing best period for processing

## Visuailzation

### Time series and their histograms

In [ ]:
from src.em.monitor import show_genral_info

show_genral_info(
    series=data_clean["BX"],
    add_title='Пропуски "склеены".',
    add_xaxis=data_clean["ydhm_id"],
)

### 3D histograms for time series

Here is code for 3D histogram visualization. It is implemented into 
DynamicMixture class as you will see later.

In [ ]:
def hist3D(data, window_size, bins, step):
    def construct_hist3D(data, window_size, bins, step):
        """
        Функция составляет 3D гистограммы.

        Параметры
        ----------
        data : pandas.core.series.Series
            Колонка таблицы pandas.core.frame.DataFrame.
        window_size : int
            Длина поднабора (окна) data, который будет использоваться при анализе.
        bins : int
            Ширина ячейки гистограммы.
        step : int
            Величина смещения окна, относительно предыдущего.

        Возвращает:
        ----------
        df: pandas.core.frame.DataFrame
            Таблица с координатами точек привязок: bins, wind_numb, -
            и высотами столбцов - hist_freq.
            df.attrs: dict
                Содержит вспомогательную информацию, используемую при кастомизации.
        """
        from pandas import DataFrame
        from numpy import histogram, meshgrid

        num_windows = len(data) - window_size + 1
        # file_save_name = f"{data.name}-l{len(data)}-ws{window_size}-s{step}-b{bins}"
        df = DataFrame({"bins": [], "wind_numb": [], "hist_freq": []})

        df.attrs = {
            "data_name": data.name,
            "data_length": len(data),
            "window_size": window_size,
            "step_size": step,
            "bin_size": bins,
        }

        for i in range(0, num_windows, step):
            window = data[i : i + window_size]
            hist, _bins = histogram(window, bins=bins)
            xpos, ypos = meshgrid(_bins[:-1], i)
            xpos = xpos.flatten()
            ypos = ypos.flatten()
            dz = hist.flatten()
            df.loc[len(df.index) - 1] = [xpos, ypos, dz]

        return df

    def visualize_3D_hist(hist3D):
        """
        Строит объемный график, представляющий динамику изменения гистограмм в
        зависимости от положения окна. Сечение, перпендикулярное оси y "№ Окна" -
        это гистограмма в соответсвующем окне.
        """
        import plotly.graph_objects as go

        # Выделение данных
        x = hist3D["bins"].values
        y = hist3D["wind_numb"].values
        z = hist3D["hist_freq"].values

        # Построение 3D поверхности
        fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

        # Персонализация изолиний и проекции
        custom_contours_z = dict(
            show=True, usecolormap=True, highlightcolor="limegreen", project_z=True
        )
        fig.update_traces(contours_z=custom_contours_z)

        # Персонализация осей
        custom_scene = dict(
            xaxis=dict(title="Интервалы гист-ы", color="grey"),
            yaxis=dict(title="№ Окна", color="grey"),
            zaxis=dict(
                title="Приращения " + hist3D.attrs.get("data_name") + ", нТ",
                color="grey",
            ),
        )

        # Название графика
        custom_title = (
            f"Компонента: {hist3D.attrs.get('data_name')}, "
            f"кол-во данных: {hist3D.attrs.get('data_length')}, "
            f"размер окна: {hist3D.attrs.get('window_size')}, "
            f"кол-во интервалов {hist3D.attrs.get('bin_size')}, "
            f"длина шага: {hist3D.attrs.get('step_size')}."
        )

        # Персонализация графика
        fig.update_layout(
            title=custom_title,
            scene=custom_scene,
            autosize=True,
            width=1200,
            height=600,
            margin=dict(l=65, r=50, b=65, t=90),
        )
        return fig

    return visualize_3D_hist(construct_hist3D(data, window_size, bins, step))


hist3D(dataframe["Bx"][:9000], window_size=4300, step=30, bins=40).show()
hist3D(dataframe["By"][:9000], window_size=4300, step=30, bins=40).show()
hist3D(dataframe["Bz"][:9000], window_size=4300, step=30, bins=40).show()

# ЕМ-algorithm

### Desctiption

* __Е-step__

    1. Calculate unnormalized responsibilities: 
    $$
    \normalsize{ \quad \tilde\rho_k^{[i]} = \pi_k \cdot \frac{1}{\sigma_k \sqrt{2\pi}} \cdot \exp{\left(-\frac{(x^{[i]} - \mu_k)^2}{2\sigma^2}\right)} \equiv
    \pi_k \cdot \frac{1}{\sigma_k} \cdot \varphi \left(\frac{x^{[i]} - \mu_k}{2\sigma} \right) }
    $$
    2. Normilize responsibilities: 
    $$
    \normalsize{ \quad \rho_k^{[i]} = \frac{\tilde\rho_k^{[i]}}{\sum_{k=0}^{M-1} \tilde\rho_k^{[i]}} }
    $$
    3. Calculate class responsibilities: 
    $$
    \normalsize{ \quad \gamma_k = \sum_{i=0}^{N-1} \rho_k^{[i]} }
    $$
    
* __М-step__

    1. Update the class probabilities: 
    $$
    \normalsize{ \quad \pi_k = \frac{\quad \gamma_k}{N} }
    $$
    2. Update the math. expectations: 
    $$
    \normalsize{ \quad \mu_k = \frac{1}{\gamma_k} \cdot \sum_{i=0}^{N-1} \rho_k^{[i]}x^{[i]} }
    $$
    3. Update the standard deviations:
    $$
    \normalsize{ \quad \sigma_k = \sqrt{\frac{1}{\gamma_k} \cdot \sum_{i=0}^{N-1} \rho_k^{[i]}\left(x^{[i]} - \mu_k \right)^2} }
    $$

### Generate mixture, check EMs on valid initial parameters, compare results 

In [ ]:
# Create mixture model
m = Mixture(num_comps=3, distrib=tensorflow_probability.distributions.Normal)
np.random.seed()
rseed = np.random.randint(1_000)
m.initialize_probs_mus_sigmas(random_seed=rseed)
print(f"radnom seed equal to {rseed}")
# Generate data for testing
m.generate_samples(1_000, random_seed=57)
# another_data = m.construct_tpf_mixture().sample(1_000).numpy()
test_data1 = m.samples
# test_data2 = another_data


## additional stuff; output appearancece
def __round(numb, dig=4):
    return round(numb, dig)


def __Round(arr, dig=4):
    return list(map(lambda numb: round(numb, dig), arr))


def custom_print(text, *args, func=__Round):
    print(
        text,
        f"Orig - {func(args[0])}",
        f"Iter - {func(args[1])}",
        f"Adap - {func(args[2])}",
        sep="\n\t",
    )


# EM comparison
pit, mit, sit, llhit = m.EM_iterative(test_data1, 30)
pad, mad, sad, llhad = m.EM_adaptive(test_data1, 0.001)

custom_print("Components probabilities:", m.probs, pit, pad)
custom_print("Mathematical expectations:", m.mus, mit, mad)
custom_print("Standard deviations:", m.sigmas, sit, sad)
custom_print(
    "Log-likelihoods:", m.log_likelihood(test_data1), llhit, llhad, func=__round
)

#### Checking sieving EM

In [ ]:
np.random.seed()
rseed = np.random.randint(1_000)
m = Mixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    random_seed=rseed,
    rand_initialize=True,
)

print(f"random seed equal to {rseed}")
# Generate data for testing
m.generate_samples(1_000, random_seed=rseed)


def sortAndRound(arr, n):
    return [round(elem, n) for elem in np.sort(arr)]


# print("Original probs:",round_sort(m.probs, 4),'\n')
res = m.EM_sieving(
    m.samples,
    iter_initial=8,
    num_candid=50,
    num_best_candid=10,
    accur_final=0.001,
    random_seed=rseed,
)

print("Original mixture:", sortAndRound(m.probs, 4), m.mus, m.sigmas, sep="\n")
print("Sieved EM result:", *res, sep="\n")
orig_mat = np.array([m.probs, m.mus, m.sigmas])
pred_mat = np.array([res[0], res[1], res[2]])
print("\nNorma for not sorded matrixes: ", np.linalg.norm(pred_mat - orig_mat))
print(
    "\nDistance between originals and predictions - ",
    round(np.linalg.norm(np.sort(m.__getattribute__("probs")) - np.sort(res[0])), 4),
)

# Reconstruction a(t) and b(t) coefficients as a time function

$\textbf{Задача 6}$. Реконструировать коэффициенты $a(t)$ и $b(t)$, то есть построить их «точечные» оценки
путем использования оценок распределений коэффициентов уравнения (1), полученных в резуль-
тате решения Задач 1 – 2, для построения оценок самих коэффициентов. В качестве таких оценок
берутся математическое ожидание и среднеквадратическое отклонение оцененного распределе-
ния:
$$
a(t) ≈ a(t) = \sum_{k=1}^{K}{p_k a_k}, \quad 
b(t) ≈ b(t) = \sum_{k=1}^{K}{p_k\cdot(b^{2}_k + a^{2}_k) − a(t)^2}
$$
Здесь t – время (положение окна), параметры $a_k , b_k , p_k$ также зависят от положения окна.

## For dBX only

In [ ]:
COMPNUM = 3

STEP = 60
SIZE = 4300

TIME = dataframe["ydhm_id"].values[1:]
dbx = dataframe["dBX"].values[1:]

mix_dBX = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

mix_dBX.predict_light(data=dbx)
a, b = mix_dBX.reconstruct_process_coef()
line(a)

In [ ]:
import numpy as np
from scipy.optimize import leastsq
import matplotlib.pyplot as plt

data = mix_dBX.process_coefs["a"]
N = 1000  # number of data points
t = np.linspace(0, len(data), N)
t = np.array(range(len(data)))
# f = 1.15247 # Optional!! Advised not to use

guess_mean = np.mean(data)
guess_std = 1
guess_phase = 0
guess_freq = 0.01
guess_amp = 3

# we'll use this to plot our first estimate. This might already be good enough for you
data_first_guess = guess_std * np.sin(t + guess_phase) + guess_mean


# Define the function to optimize, in this case, we want to minimize the difference
# between the actual data and our "guessed" parameters
def optimize_func(x, data):
    return x[0] * np.sin(x[1] * t + x[2]) + x[3] - data


params_sin = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]
est_amp, est_freq, est_phase, est_mean = params_sin

# recreate the fitted curve using the optimized parameters
data_fit = est_amp * np.sin(est_freq * t + est_phase) + est_mean

# recreate the fitted curve using the optimized parameters
fine_t = t  # np.arange(0,max(t),0.1)
data_fit0 = est_amp * np.sin(est_freq * fine_t + est_phase) + est_mean

plt.plot(t, data, ".")
# plt.plot(t, data_first_guess, label='first guess')
plt.plot(fine_t, data_fit, label="after fitting")
plt.legend()
plt.show()

In [ ]:
line(dict(sin=data_fit, orig=data))

In [ ]:
from scipy.stats import kstest

norm_hypot_pval = kstest(data_fit - data, cdf="norm").pvalue
histogram(data_fit - data, title=f"p-value = {norm_hypot_pval}")

In [ ]:
data = mix_dBX.process_coefs["a"] - data_fit0
guess_mean = np.mean(data)
guess_std = 0.0
guess_phase = 0
guess_freq = 0.05
guess_amp = 0.03

# we'll use this to plot our first estimate. This might already be good enough for you
# data_first_guess = guess_std*np.sin(t+guess_phase) + guess_mean


# Define the function to optimize, in this case, we want to minimize the difference
# between the actual data and our "guessed" parameters
def optimize_func(x, data):
    return x[0] * np.sin(x[1] * t + x[2]) + x[3] - data


params_sin = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]
est_amp, est_freq, est_phase, est_mean = params_sin

# recreate the fitted curve using the optimized parameters
data_fit = est_amp * np.sin(est_freq * t + est_phase) + est_mean

# recreate the fitted curve using the optimized parameters
fine_t = t  # np.arange(0,max(t),0.1)
data_fit = est_amp * np.sin(est_freq * fine_t + est_phase) + est_mean

plt.plot(t, data, ".")
# plt.plot(t, data_first_guess, label='first guess')
plt.plot(fine_t, data_fit, label="after fitting")
plt.legend()
plt.show()

In [ ]:
line(dict(sin=data_fit + data_fit0, orig=data))

In [ ]:
line(dict(sin=data_fit, orig=data))

In [ ]:
from scipy.stats import kstest

norm_hypot_pval = kstest(data_fit - data, cdf="norm").pvalue
histogram(
    data_fit - data,
    title=f"p-value = {norm_hypot_pval}",
)

## For all at once

In [ ]:
COMPNUM = 3

STEP = 60
SIZE = 4300

TIME = dataframe["ydhm_id"].values[1:]

dbx_mixture = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

dby_mixture = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

dbz_mixture = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

vx_mixture = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

vy_mixture = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

vz_mixture = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)

MIXTURES = [dbx_mixture, dby_mixture, dbz_mixture, vx_mixture, vy_mixture, vz_mixture]

dbx = dataframe["dBX"].values[1:]
dby = dataframe["dBY"].values[1:]
dbz = dataframe["dBZ"].values[1:]

vx = dataframe["Vx_Velocity"].values[1:]
vy = dataframe["Vy_Velocity"].values[1:]
vz = dataframe["Vz_Velocity"].values[1:]

DATA = [dbx, dby, dbz, vx, vy, vz]

In [ ]:
dvx = dataframe["dVX"].values[1:]
dvy = dataframe["dVY"].values[1:]
dvz = dataframe["dVZ"].values[1:]

In [ ]:
for mixture, data in zip(MIXTURES, DATA):
    # calculate parameters
    mixture.predict_light(data=data)

In [ ]:
COEFS = []
for mixt in MIXTURES:
    a, b = mixt.reconstruct_process_coef()
    coefs = dict(a=a, b=b)

    COEFS.append(coefs)

## Visualization: pictures

In [ ]:
import plotly as plt
from plotly.express import line

In [ ]:
var_name = ["dBX", "dBY", "dBZ", "VX", "VY", "VZ"]

for coefs, name in zip(COEFS, var_name):
    fig = line(
        coefs,
        title=name + " | dX(t) = a(t) dt + b(t) dW\t" + "| Window length=4500, step=60",
    )
    fig.show()

In [ ]:
from os.path import join

var_name = ["dBX", "dBY", "dBZ", "VX", "VY", "VZ"]
for mix, name in zip(MIXTURES, var_name):
    bold_name = f"<b>{name}</b>"
    fig = mix.show_parameters()
    fig.update_layout(
        height=1000,
        width=1200,
        title=dict(
            text=f"{bold_name} {COMPNUM}-components mixture parameters. Window size is {SIZE} and step is {STEP}",
            x=0.5,
            font=dict(size=22, color="#000000"),
        ),
    )
    file_name = f"{name}_params"
    fig.write_html(join("Figures", file_name) + ".html")

RAM is filled with useless (at this point), but heavy objects. 
So it is wise to clean it up. 

In [ ]:
for mix in MIXTURES:
    del mix

RAM is filled with useless (at this point), but heavy objects. 
So it is wise to clean it up. 

In [ ]:
for mix in MIXTURES:
    del mix

## Draft: parameters ordering ideas

In [ ]:
def procpar(pk, ak, bk):
    "for a specific window"
    a = np.sum(pk * ak)
    b = np.sum(pk * (bk**2 * ak**2) - a**2)
    return a, b


ar1 = np.array([1, 2, 3])
ar2 = np.array([7, -1, 2])
procpar(ar1, ar2, 3)

In [ ]:
def get_ind(i, dic):
    return dic["probs"][i], dic["mus"][i], dic["sigmas"][i]


res1 = get_ind(177, mixture.parameters)
res2 = get_ind(178, mixture.parameters)
res3 = get_ind(179, mixture.parameters)

In [ ]:
print(res1, "\n", res2, "\n", res3)

Assume we have a matrix of same-size lists, that contains integer numbers, that 
correspond to some order of mixture parameters.
On each neighbouring windows we want to compare those matrixes to find out if
any parameters variables crossed each other and change raletive positions. 
For this case of course basic ordering (for example by increasing of standard
diviation) not working because this approach exclude oppartunity for 
this specific parameter to have intesetions between different components.

It means that we need to think of something trickier.
Let's try implenet next approach:
1. Initialize some integer values to track order. For expamle - get indexes of
increasing sorting of each array. And sort all values by one of the rows 
(for instance by sigmas). Thereupon save this values into matrix to ceep the track on.
2. Calculate values on next window, sort by previously chosen row, 
save them in new matrix and compare them to previous 'order matrix'.
3. If those matrixes are equal, then save current order matrix into previos and
move on step 2.
4. Otherwise at list one array differes at list by two values 

(example: 
$\quad[[1,2,0][0,2,1]\textbf{[0,1,2]}] vs [[1,2,0][0,2,1] \textbf{[2,1,0]}] \quad$
, - first and last arguments differes). 

Chance of two arrays change their values at the same time is orbitrary low (I think).
So if amended array is not on the same row that we are using for sorting, than \
we're fine. We've just tracked intersection. Move on step 2.
5.  However if this array is on the same row that we are sorting by, then there 
must be applied additional actions. We need to take other row from 'previous 
order matrix'. See how it correspond to originaly chosen row, save their
'relation' and resort current order matrix's original row. Other rows shouldn't
be altered!




Also it is obvious 
that only columns may switch their places. It means them there is no use in sorting values by line
I need to think of 'vertical' ordering. For ecamle I cand calculated initial orders and make dictionary of 
three elements: 'first line", ..,'third line'. All this elements will contain a 1-D vector.
And if the values changes than columns are switching. It means that 

In [ ]:
def order(*args):
    order_matrix = []
    indexs = range(len(args[0]))

    for arg in args:
        order_matrix.append(sorted(indexs, key=lambda k: arg[k]))
    return order_matrix


def order_by(indexs, args):
    for arg in args:
        arg = np.array([arg[i] for i in indexs])
        print(arg)


def alter_order(ordmat_prev: list, ordmat_cur: list):
    from copy import deepcopy

    omp = deepcopy(ordmat_prev)
    omc = deepcopy(ordmat_cur)
    print(omp != omc)
    if omp != omc:
        print(omp, omc)
        omp.pop(-1)
        omc.pop(-1)
        if omp != omc:
            return omp.pop(-1)
        else:
            alter_order(omp, omc)
    # return omc.pop(-1)
    # if all([a_id == b_id for a_id, b_id in zip(omp, omc)]):

In [ ]:
def swapes_ones(ar1, ar2):
    pairs = []
    for el1, el2 in zip(ar1, ar2):
        if (el1 == el2) or (any([el1 in pair for pair in pairs])):
            continue
        else:
            pairs.append([el1, el2])
    return pairs


def swap(pairs, arr):
    for i, ar in enumerate(arr):
        for par in pairs:
            if ar in par:
                par.remove(ar)
                print(int(par), ar)
                arr[i] = par

# [Kolmogorov-Smironov test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html)

Might be useful:
* python [Kolmogorov-Smirnov one-sided test statistic distribution](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ksone.html#scipy.stats.ksone)    

## Static mixture

First of all let's try this test on a basic data such as static mixture.
For that reason will use `class mixture` and go through next steps:
1. Generate samples for a mixture
2. Shuffle all it's values and separete them into testing and validational
sets
3. Apply EM algorithm onto testing data to calculate mixture parameters and
calcualte with `mixture.construct_tpf_mixture` [cumulative distribution function](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MixtureSameFamily#cdf) 
for a mixture 
4. Run K.S. test on validation data and cdf 

In [ ]:
vx_mixture = DynamicMixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=dataframe["ydhm_id"].values,
    window_shape=(4500, STEP),
)

vx = dataframe["Vx_Velocity"].values
vx_mixture.windows

In [ ]:
vx_mixture.rewrite_as_normal_human_this_initialization(42)

In [ ]:
vx_mixture.predict_ks(vx, train_perc=0.5, relprev_pos=1)

In [ ]:
from scipy.stats import kstest
import tensorflow_probability


## additional stuff; output appearancece
def custom_print(text, *args, func=__Round):
    print(text, f"Orig - {func(args[0])}", f"Iter - {func(args[1])}", sep="\n\t")

In [ ]:
""" 1 """
# Create mixture model
m = Mixture(num_comps=3, distrib=tensorflow_probability.distributions.Normal)
# np.random.seed()
rseed = np.random.randint(1_000)
m.initialize_probs_mus_sigmas(random_seed=rseed)
print(f"radnom seed equal to {rseed}")
# Generate data for testing
m.generate_samples(1_000, random_seed=57)

samples = m.samples.copy()

""" 2 """
np.random.shuffle(samples)
test_perc = 0.6
test_size = int(test_perc * len(samples))

test = samples[:test_size]
valid = samples[test_size:]

""" 3 """
pit, mit, sit, llh = m.EM_iterative(test, 30)
tpf_m_on_test = mixture.construct_tpf_mixture(pit, mit, sit)
# # EM comparison
# pit, mit, sit, llhit = m.EM_iterative(samples, 30)

""" 4 """
m_test = Mixture(
    num_comps=m.num_comps,
    distrib=m.distrib,
    comp_probs=pit,
    math_expects=mit,
    stand_devs=sit,
)


def mixture_cdf(x):
    return tpf_m_on_test.cdf(x).numpy()


kolmog_test = kstest(valid, mixture_cdf)

custom_print("Components probabilities:", m.probs, pit)
custom_print("Mathematical expectations:", m.mus, mit)
custom_print("Standard deviations:", m.sigmas, sit)
# custom_print("Log-likelihoods:", m.log_likelihood(test_data1), llhit,
#              func=_round)

In [ ]:
pks, mks, sks, lks = m.Kolmogorov_EM(m.samples, relprev_pos=1, random_seed=80)
custom_print("Components probabilities:", m.probs, pks)
custom_print("Mathematical expectations:", m.mus, mks)
custom_print("Standard deviations:", m.sigmas, sks)

In [ ]:
from src.em.algorithms import KS_test

KS_test(valid, probs=pit, mus=mit, sigmas=sit)

# Trigonometric approximation

In [ ]:
data = pd.read_csv("full_data.csv")
data.columns

`Code to read from 1min_csv folder`
<!-- 
path = 'Data_nice/1min_csv'
files = os.listdir(path=path)
files.remove('00readme.txt')
files.sort()
print(files)
dfs = []
df = pd.read_csv(
    filepath_or_buffer=os.path.join(path, files[0]),
    na_values=['-1.00E+31','1.00E+31'],
    converters={12: float, 13: float, 14:float, 15:float}
    )
dfs.append(df)
for filename in files[1:]:
    df = pd.read_csv(
        filepath_or_buffer=os.path.join(path, filename),
        na_values=['-1.00E+31','1.0E+31', '-1e+31'],
        converters={12: float, 13: float, 14:float, 15:float}
    )
    dfs.append(df)
data = pd.concat(dfs, axis=0, ignore_index=True)
def tri(df):
    # Создание столбца 'ydhm_id'
        # Функция для преобразования значения к нужному виду
    def format_value(value):
        return f'{value:02}'
    
    make_cell = lambda row: str(
        f"{row['Year']:02}"+
        f"-{row['Month']:02}"+
        f"-{row['Day']:02}"
        f"T{row['Hour']:02}"+
        f":{row['Min']:02}")
    df['ydhm_id'] = df.apply(make_cell, axis=1) -->


In [ ]:
gaps = []
for i, elem in enumerate((data["BX_GSE"].values)):
    if pd.isna(elem):
        gaps.append(i)


def omit_gaps(ar: list, dif=3):
    s0 = ar[0]
    newar = []
    bad = []
    for space in ar[1:]:
        if space - s0 <= dif:
            bad.append(space)
            if abs(ar[-1] - space) <= dif:
                ar.pop()
                ar.append(f"bad iters from {space}")
        else:
            newar.append(space)
        s0 = space
    return newar, bad


g1, b1 = omit_gaps(gaps, 2)

`Gaps visualization`
<!-- fig = make_subplots(
    rows=1, 
    cols=1)

fig.add_trace(
    go.Scatter(
        x=b1, 
        y=[1]*len(b1),
        name='bad',
        mode='markers'
    ), 
    row=1, 
    col=1
    )

fig.add_trace(
    go.Scatter(
        x=g1, 
        y=[1.1]*len(g1),
        name='good',
        mode='markers'
    ), 
    row=1, 
    col=1
    ) -->

In [ ]:
start = 196853
stop = 279681

In [ ]:
bx = data["BX_GSE"][start:stop].values
# by = data['BY_GSE'][start:stop].values
# bz = data['BZ_GSE'][start:stop].values
time = data["ydhm_id"][start:stop].values

In [ ]:
def fill_gaps(data, neighs=2):
    for i, val in enumerate(data):
        if pd.isna(val):
            data[i] = np.mean(
                bx[i - neighs : i + neighs][~np.isnan(bx[neighs - 2 : neighs + 2])]
            )


fill_gaps(bx)

# fill_gaps(by)
# fill_gaps(bz)
# for i, val in enumerate(by):
#     if pd.isna(val):
#         print(i,"Lnlj")

# line(dict(bx=bx, by=by, bz=bz), x=time, y=['bx', 'by', 'bz'])

In [ ]:
def increm(arr):
    new_ar = [None]
    for i in range(1, len(arr)):
        inc = arr[i] - arr[i - 1]
        new_ar.append(inc)
    return new_ar


def smooth(data, wind_size=20):
    from numpy.lib.stride_tricks import sliding_window_view
    from numpy import mean

    windows = sliding_window_view(data, wind_size)
    smoothed = []
    for wind in windows:
        smoothed.append(mean(wind))
    return smoothed


dbx = increm(bx)[1:]

# dby = increm(by)[1:]
# dbz = increm(bz)[1:]

In [ ]:
COMPNUM = 5

STEP = 60
SIZE = 4300
TIME = time[int(SIZE / 2) : len(dbx) - int(SIZE / 2) : STEP]

mix_dBX = DynamicMixture(
    num_comps=COMPNUM,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=TIME,
    window_shape=(SIZE, STEP),
)
mix_dBX.predict_light(data=dbx)
ax, bx = mix_dBX.reconstruct_process_coef()

# mix_dBY = DynamicMixture(
#     num_comps=COMPNUM,
#     distrib=tensorflow_probability.distributions.Normal,
#     time_span=TIME,
#     window_shape=(SIZE, STEP)
#     )
# mix_dBY.predict_light(data=dby)
# ay,by = mix_dBY.reconstruct_process_coef()

# mix_dBZ = DynamicMixture(
#     num_comps=COMPNUM,
#     distrib=tensorflow_probability.distributions.Normal,
#     time_span=TIME,
#     window_shape=(SIZE, STEP)
#     )
# mix_dBZ.predict_light(data=dbz)
# az,bz = mix_dBZ.reconstruct_process_coef()

In [ ]:
def harmonic_approximation(data, time, harmonics_num=4, title=""):
    import numpy as np
    from scipy.optimize import leastsq
    import plotly.graph_objects as go
    from scipy.stats import kstest, norm
    from plotly.subplots import make_subplots

    def find_frequency(data, sampling_rate):
        n = len(data)
        # t = np.arange(0, n) / sampling_rate
        fft_result = np.fft.fft(data)
        freqs = np.fft.fftfreq(n, d=1 / sampling_rate)
        spectrum = abs(fft_result)

        idx = np.argmax(spectrum[1:]) + 1  # Избегаем нулевую частоту
        freq = freqs[idx]

        return abs(freq)

    fig = make_subplots(rows=3, cols=1)
    data_orig = data
    t = np.array(range(len(data)))
    params = []
    harmonical_signal = np.zeros(len(data))

    for _ in range(harmonics_num):
        guess_mean = np.mean(data)
        guess_phase = 0
        guess_freq = find_frequency(data, len(data)) * np.pi * 2 / len(data)
        guess_amp = max(data) - min(data)

        def optimize_func(x):
            return x[0] * np.sin(x[1] * t + x[2]) + x[3] - data

        params_sin = leastsq(
            optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean]
        )[0]

        params.append(params_sin)
        est_amp, est_freq, est_phase, est_mean = params_sin

        data_fit = est_amp * np.sin(est_freq * t + est_phase) + est_mean
        data = data - data_fit
        harmonical_signal += data_fit

    # Approximation visualization
    fig.add_trace(
        go.Scatter(x=time, y=data_orig, marker=dict(color="#3058B0")),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(x=time, y=harmonical_signal, marker=dict(color="#FF7F50")),
        row=1,
        col=1,
    )
    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.5,
        y=1.2,
        showarrow=False,
        font=dict(size=22),
        text="<b>Approximation<b>",
        row=1,
        col=1,
    )

    # Residuals
    fig.add_trace(
        go.Scatter(x=time, y=data, marker=dict(color="#FF7F50")), row=2, col=1
    )
    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.5,
        y=1.2,
        showarrow=False,
        font=dict(size=22),
        text="<b>Residuals<b>",
        row=2,
        col=1,
    )

    # Histogramm
    fig.add_trace(
        go.Histogram(
            x=data, histnorm="probability density", marker=dict(color="#3058B0")
        ),
        row=3,
        col=1,
    )
    x = np.linspace(min(data) * 1.2, max(data) * 1.2, 100)
    fig.add_trace(
        go.Scatter(x=x, y=norm.pdf(x, *norm.fit(data)), marker=dict(color="#FF7F50")),
        row=3,
        col=1,
    )

    def cdf(x):
        return norm.cdf(x, loc=norm.fit(data)[0], scale=norm.fit(data)[1])

    pval = kstest(data, cdf=cdf).pvalue
    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.5,
        y=1.3,
        showarrow=False,
        font=dict(size=22),
        text="<b>Residuals histogram<b>",
        row=3,
        col=1,
    )
    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.5,
        y=1.2,
        showarrow=False,
        font=dict(size=20),
        text=f"Kolmogorov-Smirnov p-value = {pval:.3f}",
        row=3,
        col=1,
    )

    fig.update_layout(
        width=1500, height=1200, title=dict(font=dict(size=20), text=f"<b>{title}<b>")
    )
    return fig, data, params

In [ ]:
def norm_test(resid, pvalue=0.05, smw=(50, 1)):
    from numpy.lib.stride_tricks import sliding_window_view
    from scipy.stats import kstest, norm

    ws = sliding_window_view(resid, smw[0])[:: smw[1]]
    P_vals = []
    for frame in ws:

        def cdf(x):
            return norm.cdf(x, loc=norm.fit(frame)[0], scale=norm.fit(frame)[1])

        pval = kstest(frame, cdf=cdf).pvalue
        P_vals.append(pval)
        if pval <= pvalue:
            raise ValueError("No normal distribution on windows occure")
    return line(dict(pvalue=P_vals, significance_level=[pvalue] * len(P_vals)))

In [ ]:
# line(dict(ax=ax, ay=ay, az=az), x=TIME, y=['ax','ay', 'az'])

In [ ]:
a_dbx_smoothed = smooth(mix_dBX.process_coefs["a"], wind_size=30)

In [ ]:
fig_x_smoothed, resid_x_smoothed, harmcs_x_smoothed = harmonic_approximation(
    a_dbx_smoothed, TIME, title="dBX GSE", harmonics_num=9
)
fig_x_smoothed.show()

In [ ]:
def approx(params_harms, data, freq_scale=1.0):
    t = np.array(range(len(data)))
    harmonical_signal = np.zeros(len(data))

    for params_sin in params_harms:
        est_amp, est_freq, est_phase, est_mean = params_sin
        est_freq *= freq_scale
        data_fit = est_amp * np.sin(est_freq * t + est_phase) + est_mean
        data = data - data_fit
        harmonical_signal += data_fit
    return data, harmonical_signal

In [ ]:
d, h = approx(
    harmcs_x_smoothed,
    mix_dBX.process_coefs["a"],
    freq_scale=(len(resid_x_smoothed) / len(mix_dBX.process_coefs["a"])),
)

In [ ]:
line({"data": mix_dBX.process_coefs["a"], "harms": h})

In [ ]:
norm_test(d, smw=(120, 1))

In [ ]:
fig_x, resid_x, harmcs_x = harmonic_approximation(
    mix_dBX.process_coefs["a"], TIME, title="dBX GSE", harmonics_num=5
)
fig_x.show()

# Drafts in general

In [ ]:
def smooth(data, wind_size=20):
    from numpy.lib.stride_tricks import sliding_window_view
    from numpy import mean

    windows = sliding_window_view(data, wind_size)
    smoothed = []
    for wind in windows:
        smoothed.append(mean(wind))
    return smoothed

$$
\mathbb{D}Z = \mathbb{D}V + \mathbb{E}U^2
\newline
\mathbb{E}U^2 = \sum_{j=1}^{k}{\sigma_j^2 \cdot p_j} - 
    \text{диффузионная компонента}
\newline
\mathbb{D}V = \sum_{j=1}^{k}{(\mu_j - \bar{\mu}) \cdot p_j}, \quad \text{где} 
    \quad \bar{\mu} = \sum_{j=1}^{k}{\mu_j p_j}, - 
    \text{динамическая компонента}
$$